In [146]:
import requests
import urllib.parse
from bs4 import BeautifulSoup
import re

In [148]:
url='https://www.jobbank.gc.ca/jobsearch/jobsearch?flg=E&fcan=1&fss=1&fss=C&fwcl=B&fwcl=C&fwcl=D&fwcl=E&fglo=1&sort=M&fwht=D&fwht=M&fsrc=16&fjnc=1&fexp=0&fexp=1&fprov=AB&fprov=ON&fskl=101020&fskl=101010'
page=requests.get(url)

soup=BeautifulSoup(page.content, 'html.parser')

job_elements=soup.find_all('a', class_='resultJobItem')

print(job_elements)

[<a class="resultJobItem" href="/jobsearch/jobposting/41460166;jsessionid=07737363B3770569D6153B794F9BE064.jobsearch77?source=searchresults" id="ajaxupdateform:j_id_34_3_3p:0:j_id_34_3_3r">
<h3 class="title">
<span class="flag">
<span class="new">
						New
					</span>
</span>
<span class="job-source job-source-icon-16"><span class="wb-inv">Job Bank</span></span>
<span class="noctitle"> guard, security
				
					<span class="job-marker">
<span class="verified">
<span aria-hidden="true" class="fa-stack">
<span class="fas fa-circle fa-stack-2x"></span>
<span class="fas fa-check fa-stack-1x fa-inverse fa-icon"></span>
</span>
<span class="wb-inv">Verified</span>
</span>
<span class="description"><span aria-hidden="true" class="fa fa-info-circle"></span>This job was posted directly by the employer on Job Bank.</span>
</span>
</span>
</h3>
<ul class="list-unstyled">
<li class="date">July 15, 2024
			</li>
<li class="business">Central Protection Services Inc.</li>
<li class="location"><span

In [196]:
role_list=[]
dates_list=[]
business_list=[]
locations_list=[]
salaries_list=[]
id_list=[]

jobs={
    
    'ID':id_list,
    'ROLES':role_list,
    'SALARY':salaries_list,
    'EDUCATION':education_list,
    'EXPERIENCE':experience_list,
    'NOC CODE':noccode_list,
    'BUSINESS':business_list,
    'LOCATIONS':locations_list,
    'PUBLISHED DATES':dates_list,
    'EMPLOYMENT TYPE':employment_type_list,
    'JOB DURATION':job_duration_list,
    
}

for job_list in job_elements:
    
    role=job_list.find('span', class_='noctitle')
    fixed_role=role.find(string=True, recursive=False).strip() #Cuando usas Recursive, no pone los datos de los demás "span".
    role_list.append(fixed_role)
    
    published_date=job_list.find('li', class_='date')
    fixed_date=published_date.find(string=True, recursive=False).strip() 
    dates_list.append(fixed_date)

    business_name=job_list.find(class_='business')
    fixed_business=business_name.find(string=True, recursive=False).strip()
    business_list.append(fixed_business)

    location=job_list.find('li', class_='location').get_text()
    fixed_location=location[10:].strip()
    locations_list.append(fixed_location)

    salary=job_list.find(class_='salary').get_text()
    fixed_salary=salary[30:].strip()
    if 'hourly' in fixed_salary:
        fix_sal=fixed_salary[:5]
    if 'anually' in fixed_salary:
        fix_sal=(fixed_salary[:9].replace(",",""))/2000
    salaries_list.append(f"$"+fix_sal)

for link in soup.find_all('a'): #CHANGE THIS FOR SOMETHING BETTER
    id=link.get('href')
    try:
        id.split('/')[-1:]
        id_number=id[22:30]
        try:
            id_number=int(id_number)
            id_list.append(id_number)
        except:
            pass
    except:
        pass
        
        

#print(str(dates_list)+'\n')
#print(str(role_list)+'\n')
#print(str(business_list)+'\n')
#print(str(locations_list)+'\n')
#print(str(salaries_list)+'\n')
#print(id_list)

['July 15, 2024', 'July 12, 2024', 'July 10, 2024', 'July 10, 2024', 'July 09, 2024', 'July 08, 2024', 'July 08, 2024', 'July 08, 2024', 'July 08, 2024', 'July 07, 2024', 'July 05, 2024', 'July 04, 2024', 'July 04, 2024', 'July 04, 2024', 'July 04, 2024', 'July 04, 2024', 'July 04, 2024', 'July 03, 2024', 'July 02, 2024', 'July 02, 2024', 'July 01, 2024', 'June 30, 2024', 'June 30, 2024', 'June 30, 2024', 'June 29, 2024']

['guard, security', 'farm worker, general', 'food service supervisor', 'cook', 'beef cattle farm worker', 'kitchen helper', 'supervisor, food services', 'shift manager - fast food restaurant', 'farm supervisor', 'truck dispatcher', 'delivery drivers supervisor', 'fruit farm labourer', 'long haul truck driver', 'long haul truck driver', 'cook', 'car mechanic', 'landscape worker', 'nursery worker', 'kitchen helper', 'poultry farm worker', 'kitchen helper', 'construction labourer', 'general construction labourer', 'drywall installer', 'food counter attendant']

['Centra

In [268]:
noccode_list=[]
employment_type_list=[]
job_duration_list=[]
education_list=[]
experience_list=[]

#for ids in id_list:
try:
    id_url='https://www.jobbank.gc.ca/jobsearch/jobposting/41460166'#+str(ids)
    id_page=requests.get(id_url)
    id_soup=BeautifulSoup(id_page.content, 'html.parser')
    job_details=id_soup.find('main', class_='container')

    #NOC CODE
    noccode=job_details.find('span', class_='aa_jobbank_job_noccode').get_text()
    noccode_list.append(noccode)

    #SHIFT TYPE
    employment_type=job_details.find('span', property='employmentType')
    fixed_employment_type=employment_type.find(string=True, Recursive=False).get_text()
    employment_type_list.append(fixed_employment_type)

    #JOB DURATION
    for duration in employmentType:
        job_duration=duration.get_text(strip=True) 
        job_duration_list.append(job_duration)
        break

    #WEEKLY HOURS
    weekly_hours=job_details.find('span', property_='workHours').get_text()
    print(weekly_hours)

    #FULL ADRESS
    adress=job_details.find('span', property_='streetAddress').get_text()
    adress_list.append(adress)
    print(adress)
    
    description=job_details.find('span', property='description').get_text()

    #EDUCATION
    education=re.search(r'Education: ([^.]+)', description)
    fixed_education=education.group(1)
    education_list.append(fixed_education)

    #EXPERIENCE
    experience=re.search(r'Experience: ([^.]+)', description)
    fixed_experience=experience.group(1)
    experience_list.append(fixed_experience)

except:
    pass

#print(noccode_list)
#print(employment_type_list)
#print(job_duration_list)
#print(education_list)
#print(experience_list)

In [270]:
print(jobs)

{'ID': [41460166, 41447597, 41433573, 41432480, 41426383, 41423047, 41422125, 41420737, 41420469, 41417176, 41412591, 41404305, 41403705, 41403673, 41403240, 41402614, 41400104, 41397063, 41390813, 41385980, 41381639, 41380596, 41380595, 41380594, 41376958], 'ROLES': ['guard, security', 'farm worker, general', 'food service supervisor', 'cook', 'beef cattle farm worker', 'kitchen helper', 'supervisor, food services', 'shift manager - fast food restaurant', 'farm supervisor', 'truck dispatcher', 'delivery drivers supervisor', 'fruit farm labourer', 'long haul truck driver', 'long haul truck driver', 'cook', 'car mechanic', 'landscape worker', 'nursery worker', 'kitchen helper', 'poultry farm worker', 'kitchen helper', 'construction labourer', 'general construction labourer', 'drywall installer', 'food counter attendant'], 'SALARY': ['$19.00', '$16.71', '$17.00', '$16.55', '$16.71', '$15.50', '$18.00', '$17.30', '$21.63', '$28.85', '$28.00', '$16.55', '$31.50', '$27.80', '$18.50', '$28.0